In [2]:
cd /content/drive/MyDrive/project/project

/content/drive/MyDrive/project/project


In [ ]:
# Cell 1: Install dependencies (uncomment if running in Colab)
!pip install torch transformers datasets plotly pandas scikit-learn

In [3]:
# Cell 3: Imports
import os
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from data.liar_dataset import LIARDataset
from models.model_utils import initialize_model, create_scheduler, train_model, test_model, save_model
from config.config import config

In [4]:
# Cell 4: Set up parameters and directories
model_type_bert = "Bert"
model_type_roberta = "Roberta"
epochs = config["epochs"]
batch_size = config["batch_size"]
learning_rate = config["learning_rate"]
eps = config["eps"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories for saving logs and models (update these paths for your Google Drive)
log_dir = "/content/drive/My Drive/logs"
model_save_dir = "/content/drive/My Drive/saved_models"

In [5]:
# Cell 5: Initialize datasets and dataloaders for BERT
train_dataset_bert = LIARDataset(split="train", model_type=model_type_bert)
validation_dataset_bert = LIARDataset(split="validation", model_type=model_type_bert)
test_dataset_bert = LIARDataset(split="test", model_type=model_type_bert)

train_dataloader_bert = DataLoader(train_dataset_bert, sampler=RandomSampler(train_dataset_bert), batch_size=batch_size)
validation_dataloader_bert = DataLoader(validation_dataset_bert, sampler=SequentialSampler(validation_dataset_bert), batch_size=batch_size)
test_dataloader_bert = DataLoader(test_dataset_bert, sampler=SequentialSampler(test_dataset_bert), batch_size=batch_size)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for ucsbnlp/liar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ucsbnlp/liar.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/10269 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1283 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1284 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/10269 [00:00<?, ? examples/s]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

Map:   0%|          | 0/1283 [00:00<?, ? examples/s]

Map:   0%|          | 0/1283 [00:00<?, ? examples/s]

In [6]:
# Cell 6: Initialize and train BERT model
model_bert, optimizer_bert = initialize_model(model_type_bert, learning_rate, eps, device)
scheduler_bert = create_scheduler(optimizer_bert, train_dataloader_bert, epochs)

print(f"Training {model_type_bert}:")
train_model(model_bert, train_dataloader_bert, validation_dataloader_bert, optimizer_bert, scheduler_bert, epochs, device, log_dir)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training Bert:
======== Epoch 1 / 2 ========
Training...
  Batch 40  of  642.    Elapsed: 0:00:27.
  Batch 80  of  642.    Elapsed: 0:00:53.
  Batch 120  of  642.    Elapsed: 0:01:20.
  Batch 160  of  642.    Elapsed: 0:01:47.
  Batch 200  of  642.    Elapsed: 0:02:15.
  Batch 240  of  642.    Elapsed: 0:02:43.
  Batch 280  of  642.    Elapsed: 0:03:10.
  Batch 320  of  642.    Elapsed: 0:03:38.
  Batch 360  of  642.    Elapsed: 0:04:06.
  Batch 400  of  642.    Elapsed: 0:04:33.
  Batch 440  of  642.    Elapsed: 0:05:01.
  Batch 480  of  642.    Elapsed: 0:05:29.
  Batch 520  of  642.    Elapsed: 0:05:56.
  Batch 560  of  642.    Elapsed: 0:06:24.
  Batch 600  of  642.    Elapsed: 0:06:52.
  Batch 640  of  642.    Elapsed: 0:07:20.
  Average training loss: 0.01
  Training epoch took: 0:07:21
Running Validation...
  Accuracy: 1.00
  Evaluation took: 0:00:20
======== Epoch 2 / 2 ========
Training...
  Batch 40  of  642.    Elapsed: 0:00:28.
  Batch 80  of  642.    Elapsed: 0:00:55.
  Ba

In [7]:
# Cell 7: Test BERT model
print(f"\nTesting {model_type_bert}:")
test_model(model_bert, test_dataloader_bert, device, log_dir)



Testing Bert:
Predicting labels for 1283 test sentences...
    DONE.
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1283

    accuracy                           1.00      1283
   macro avg       1.00      1.00      1.00      1283
weighted avg       1.00      1.00      1.00      1283

[[1283]]


In [8]:
# Cell 8: Save BERT model
save_model(model_bert, model_save_dir, "bert_model")


In [9]:
# Cell 9: Initialize datasets and dataloaders for RoBERTa
train_dataset_roberta = LIARDataset(split="train", model_type=model_type_roberta)
validation_dataset_roberta = LIARDataset(split="validation", model_type=model_type_roberta)
test_dataset_roberta = LIARDataset(split="test", model_type=model_type_roberta)

train_dataloader_roberta = DataLoader(train_dataset_roberta, sampler=RandomSampler(train_dataset_roberta), batch_size=batch_size)
validation_dataloader_roberta = DataLoader(validation_dataset_roberta, sampler=SequentialSampler(validation_dataset_roberta), batch_size=batch_size)
test_dataloader_roberta = DataLoader(test_dataset_roberta, sampler=SequentialSampler(test_dataset_roberta), batch_size=batch_size)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Map:   0%|          | 0/10269 [00:00<?, ? examples/s]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

Map:   0%|          | 0/1283 [00:00<?, ? examples/s]

Map:   0%|          | 0/1283 [00:00<?, ? examples/s]

In [10]:
# Cell 10: Initialize and train RoBERTa model
model_roberta, optimizer_roberta = initialize_model(model_type_roberta, learning_rate, eps, device)
scheduler_roberta = create_scheduler(optimizer_roberta, train_dataloader_roberta, epochs)

print(f"Training {model_type_roberta}:")
train_model(model_roberta, train_dataloader_roberta, validation_dataloader_roberta, optimizer_roberta, scheduler_roberta, epochs, device, log_dir)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



Training Roberta:
======== Epoch 1 / 2 ========
Training...
  Batch 40  of  642.    Elapsed: 0:00:28.
  Batch 80  of  642.    Elapsed: 0:00:57.
  Batch 120  of  642.    Elapsed: 0:01:24.
  Batch 160  of  642.    Elapsed: 0:01:53.
  Batch 200  of  642.    Elapsed: 0:02:21.
  Batch 240  of  642.    Elapsed: 0:02:49.
  Batch 280  of  642.    Elapsed: 0:03:17.
  Batch 320  of  642.    Elapsed: 0:03:45.
  Batch 360  of  642.    Elapsed: 0:04:13.
  Batch 400  of  642.    Elapsed: 0:04:41.
  Batch 440  of  642.    Elapsed: 0:05:09.
  Batch 480  of  642.    Elapsed: 0:05:37.
  Batch 520  of  642.    Elapsed: 0:06:05.
  Batch 560  of  642.    Elapsed: 0:06:33.
  Batch 600  of  642.    Elapsed: 0:07:01.
  Batch 640  of  642.    Elapsed: 0:07:29.
  Average training loss: 0.00
  Training epoch took: 0:07:30
Running Validation...
  Accuracy: 1.00
  Evaluation took: 0:00:20
======== Epoch 2 / 2 ========
Training...
  Batch 40  of  642.    Elapsed: 0:00:28.
  Batch 80  of  642.    Elapsed: 0:00:56.
 

In [11]:
# Cell 11: Test RoBERTa model
print(f"\nTesting {model_type_roberta}:")
test_model(model_roberta, test_dataloader_roberta, device, log_dir)



Testing Roberta:
Predicting labels for 1283 test sentences...
    DONE.
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1283

    accuracy                           1.00      1283
   macro avg       1.00      1.00      1.00      1283
weighted avg       1.00      1.00      1.00      1283

[[1283]]


In [12]:
# Cell 12: Save RoBERTa model
save_model(model_roberta, model_save_dir, "roberta_model")
